In [ ]:
import sys
import os
from multiprocessing.pool import ExceptionWithTraceback

import numpy as np
import xarray as xr
import matplotlib.pyplot as plt
from numpy.f2py.auxfuncs import throw_error

sys.path.append('..//')
from utils_mitgcm import open_mitgcm_ds_from_config
import pylake

In [ ]:
model = 'dummy_rectangle'
mitgcm_config, ds = open_mitgcm_ds_from_config('..//config.json', model)

In [ ]:
folder_path = os.path.dirname(mitgcm_config['datapath'])
output_folder = os.path.join(folder_path, "seiche_analysis", "potential_energy")
os.makedirs(output_folder, exist_ok=True)

In [ ]:
grid_resolution = 200
ds['YC'] = np.arange(1, len(ds['YC'])+1) * grid_resolution - grid_resolution/2
ds['XC'] = np.arange(1, len(ds['XC'])+1) * grid_resolution - grid_resolution/2
ds['YG'] = np.arange(0, len(ds['YG'])) * grid_resolution
ds['XG'] = np.arange(0, len(ds['XG'])) * grid_resolution

ds['Z'] = -1 * ds.Z

In [ ]:
mask = ds.THETA.isel(time=0).values != 0

In [ ]:
plt.figure(figsize=(10,3))
plt.imshow(ds.THETA.isel(time=-1,Z=0).where(mask[0], np.nan))
plt.gca().invert_yaxis()
plt.colorbar()

# Get temperature profile averaged over the entire lake

In [ ]:
ds['theta_nan'] = ds['THETA'].where(mask, np.nan)

In [ ]:
ds['mean_temp_profile'] = ds.theta_nan.isel(time=0) #ds.theta_nan.mean(dim=['time']).compute()

In [ ]:
ds['mean_temp_profile'].isel(Z=0).plot()

# Get buoyancy frequency N

In [ ]:
def buoyancy_freq(Temp, depth=None, g=9.81):
    '''
    Description: Calculate the buoyancy frequency (Brunt-Vaisala frequency) for a temperature profile. Copied from PyLake module and adapted to take xarray as input.
    https://github.com/eawag-surface-waters-research/PyLake/blob/main/pylake

    Parameters
    -----------
    Temp: array_like
        A numeric vector of water temperature in degrees C
    depth: array_like
        a numeric vector corresponding to the depth (in m) of the Temp measurements
    g: scalar, default: 9.81
        gravity acceleration (m/s2)

    Returns
    ----------
    n2: xarray with buoyancy frequency in units {sec^-2} and associated average lake depths (can be different from input depths).
    '''
    rho = pylake.dens0(s=0.2, t=Temp)
    numdepth = len(depth)
    rho_2 = rho.isel(Z=slice(0,numdepth-1))
    drho_dz = rho.diff('Z')/Temp.Z.diff('Z')
    rho_2["Z"] = drho_dz.Z
    n2 = g/rho_2*drho_dz
    n2["Z"] = [(a+b)/2 for a,b in zip(depth, depth[1:])]
    n2 = n2.rename({"Z":"avg_depth"})

    return n2

In [ ]:
xr_N = buoyancy_freq(ds.theta_nan, ds.Z)

In [ ]:
fig,ax = plt.subplots(1,1)
xr_N.isel(time=24,XC=150,YC=80).plot()
ax.set_yscale('log')

In [ ]:
xr_N = xr_N.interp({'avg_depth':ds.Z})

In [ ]:
xr_N.to_netcdf(os.path.join(output_folder, r'buoyancy_frequency.nc'))

xr_N = xr.open_dataset(os.path.join(output_folder, r'buoyancy_frequency.nc'))
xr_N = xr_N.__xarray_dataarray_variable__

In [ ]:
xr_N.isel(time=100, XC=15,YC=10).plot()

# Get vertical displacement

In [ ]:
def inverse_interp_1d(T_prof, T_target, Z):
    """
    T_prof: 1D array of temps along Z
    T_target: 1D array of temps at which we want to find the depth
    Z: 1D array of depths
    Returns: 1D array of same shape as T_target with interpolated Z
    """
    out = np.full_like(T_target, np.nan, dtype=float)

    # Mask valid (non-nan) values
    mask = ~np.isnan(T_prof)
    T_valid = T_prof[mask]
    Z_valid = Z[mask]

    if T_valid.size < 2:
        return out

    # Use searchsorted to locate bracketing temps
    idxs = np.searchsorted(T_valid, T_target)

    for j, idx in enumerate(idxs):
        if 0 < idx < len(T_valid):
            T1, T2 = T_valid[idx - 1], T_valid[idx]
            Z1, Z2 = Z_valid[idx - 1], Z_valid[idx]
            Tt = T_target[j]
            if T2 != T1:
                out[j] = Z1 + (Z2 - Z1) * (Tt - T1) / (T2 - T1)

    return out

In [ ]:
# Ensure Z sorted
ds = ds.sortby("Z", ascending=False)
z_profile = ds.Z             # (Z)

expanded_mean,_ = xr.broadcast(ds.mean_temp_profile, ds.theta_nan)
mean_temperature_profile = expanded_mean.rename({'Z':'Z_temp'})  # (time, Z_temp, XC, YC)

temperature_profiles_whole_lake = ds.theta_nan.chunk({'time': 1, 'XC':30, 'YC':30})  # (time, Z, XC, YC)

z_corresponding = xr.apply_ufunc(
    inverse_interp_1d,
    temperature_profiles_whole_lake,                # T_prof: (time, Z, XC, YC)
    mean_temperature_profile,      # T_target: (time, Z_temp, XC, YC)
    z_profile,                   # Z: (Z)
    input_core_dims=[["Z"], ["Z_temp"], ["Z"]],
    output_core_dims=[["Z_temp"]],
    vectorize=True,
    dask="parallelized",
    output_dtypes=[float],
)
z_corresponding = z_corresponding.rename({'Z_temp':'Z'})

In [ ]:
mean_temperature_profile.isel(XC=150,YC=80).plot()
temperature_profiles_whole_lake.isel(time=48,XC=150,YC=80).plot()

In [ ]:
vertical_displacement = z_profile - z_corresponding

In [ ]:
vertical_displacement.to_netcdf(os.path.join(output_folder, r'vertical_displacement.nc'))

In [ ]:
vertical_displacement.isel(XC=15,YC=10).plot()
plt.gca().set_ylim(top=40)

vertical_displacement = xr.open_dataset(r'/home/leroquan@eawag.wroot.emp-eaw.ch/work_space/dummy_extended/seiche_analysis/vertical_displacement.nc')
vertical_displacement = vertical_displacement.__xarray_dataarray_variable__

# Compute potential energy

In [ ]:
rho = 1000
surface_cell = 200 * 200
val = (vertical_displacement ** 2) * xr_N * ds.drF

In [ ]:
E_pot_profile = 0.5 * rho * surface_cell * val.sum(dim='Z') / 1e6

In [ ]:
E_pot_profile.to_netcdf(os.path.join(output_folder, r'potential_energy.nc'))

In [ ]:
plt_obj = E_pot_profile.isel(time=100).plot(vmin=0, vmax=100)
plt_obj.colorbar.set_label('Potential Energy [MJ]')

In [ ]:
total_potential_energy = E_pot_profile.sum(['XC','YC'])

In [ ]:
df_pe = total_potential_energy.to_dataframe(name='pe_mj_seiche')['pe_mj_seiche']

In [ ]:
fig, ax = plt.subplots(1,1,figsize=(10,5))
df_pe.plot()
plt.ylabel('Potential Energy [MJ]')
plt.xlabel('')
fig.savefig(os.path.join(output_folder, "potential_energy.png"))

In [ ]:
df_pe.reset_index().to_csv(os.path.join(output_folder, "potential_energy.csv"))

In [ ]:
os.makedirs(os.path.join(output_folder, 'e_pot_map'), exist_ok=True)
for i in range(25,E_pot_profile.sizes['time']):
    plt.close('all')
    plt.figure(figsize=(15,7))
    plt_obj = E_pot_profile.isel(time=i).plot(vmin=0, vmax=100)
    plt_obj.colorbar.set_label('Potential Energy [MJ]')
    plt.title(E_pot_profile.time.isel(time=i).values)
    plt.savefig(os.path.join(output_folder, 'e_pot_map', rf'potential_energy_time{i}.png'))